In [9]:
%load_ext bigdata

The bigdata extension is already loaded. To reload it, use:
  %reload_ext bigdata


In [10]:
%hive_start

In [11]:
%timeout 300

In [12]:
## Se elimina el directorio de salida en el hdfs si existe
!hadoop fs -rm input/*
!hadoop fs -rm output/*
!hadoop fs -rm output2/*
!hadoop fs -rmdir input output
!hadoop fs -rmdir input output2

rm: `input/*': No such file or directory
rm: `output/*': No such file or directory
rm: `output2/*': No such file or directory
rmdir: `input': No such file or directory
rmdir: `output': No such file or directory
rmdir: `input': No such file or directory
rmdir: `output2': No such file or directory


In [13]:
!hadoop fs -put *.tsv .
!hadoop fs -ls

put: `data.tsv': File exists
Found 1 items
-rw-r--r--   1 root supergroup        637 2020-01-19 21:10 data.tsv


In [14]:
!rm -rf input
!rm -rf output
!mkdir input
!mkdir output

In [15]:
%%writefile script.hive

## Borra la carpeta si existe
!hdfs dfs -rm -r -f /tmp/output

##
## Crea la carpeta drivers en el HDFS
##
!hdfs dfs -mkdir /tmp/output

##
## Copia los archivos al HDFS
##
!hdfs dfs -copyFromLocal data.tsv  /tmp/output/data.tsv

## Carga de datos
## Punto 1

DROP TABLE IF EXISTS data;

CREATE TABLE data (letra       STRING,
                           fecha        STRING,
                           numero      INT)

ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t';

SHOW TABLES;
SHOW CREATE TABLE data;

DESCRIBE data;

SELECT * FROM data LIMIT 5;

select * FROM data
group by letra
order by junto;

##
## Lista los archivos al HDFS para verificar
## que los archivos fueron copiados correctamente.
##
!hdfs dfs -ls /tmp/output/*

Overwriting script.hive


In [16]:
!hive -execute 'run script.hive'

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]

Logging initialized using configuration in jar:file:/usr/local/hive/lib/hive-common-2.3.6.jar!/hive-log4j2.properties Async: true
NoViableAltException(24@[])
	at org.apache.hadoop.hive.ql.parse.HiveParser.statement(HiveParser.java:1300)
	at org.apache.hadoop.hive.ql.parse.ParseDriver.parse(ParseDriver.java:208)
	at org.apache.hadoop.hive.ql.parse.ParseUtils.parse(ParseUtils.java:77)
	at org.apache.hadoop.hive.ql.parse.ParseUtils.parse(ParseUtils.java:70)
	at org.apache.hadoop.hive.ql.Driver.compile(Driver.java:468)


In [5]:
%%hive
DROP TABLE IF EXISTS data;

CREATE TABLE data (letra       STRING,
                           fecha        STRING,
                           numero      INT)

ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t';


DROP TABLE IF EXISTS data;
OK
Time taken: 13.517 seconds
CREATE TABLE data (letra       STRING,
                           fecha        STRING,
                           numero      INT)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t';
OK
Time taken: 1.331 seconds


In [6]:
%%hive
SHOW TABLES;
SHOW CREATE TABLE data;

SHOW TABLES;
OK
data
Time taken: 0.207 seconds, Fetched: 1 row(s)
SHOW CREATE TABLE data;
OK
CREATE TABLE `data`(
  `letra` string, 
  `fecha` string, 
  `numero` int)
ROW FORMAT SERDE 
  'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe' 
WITH SERDEPROPERTIES ( 
  'field.delim'='\t', 
  'serialization.format'='\t') 
STORED AS INPUTFORMAT 
  'org.apache.hadoop.mapred.TextInputFormat' 
OUTPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION
  'hdfs://0.0.0.0:9000/user/hive/warehouse/data'
TBLPROPERTIES (
  'transient_lastDdlTime'='1579227316')
Time taken: 0.38 seconds, Fetched: 17 row(s)


In [7]:
%%hive
LOAD DATA INPATH '/tmp/output/data.tsv' OVERWRITE
INTO TABLE data;

LOAD DATA INPATH '/tmp/output/data.tsv' OVERWRITE
INTO TABLE data;
Loading data to table default.data
OK
Time taken: 1.873 seconds


In [8]:
%%hive
DESCRIBE data;

DESCRIBE data;
OK
letra               	string              	                    
fecha               	string              	                    
numero              	int                 	                    
Time taken: 0.069 seconds, Fetched: 3 row(s)


In [9]:
%%hive
SELECT * FROM data LIMIT 5;

SELECT * FROM data LIMIT 5;
OK
B	1999-08-28	14
E	1999-12-06	12
E	1993-07-21	17
C	1991-02-12	13
E	1995-04-25	16
Time taken: 2.282 seconds, Fetched: 5 row(s)


In [10]:
%%hive
select concat(letra,',', count(*)) as junto from data
group by letra
order by junto;

select concat(letra,',', count(*)) as junto from data
group by letra
order by junto;
Query ID = root_20200117021553_c27e6023-c9a9-4b71-985a-3fa56e6aa3a2
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1579225114203_0001, Tracking URL = http://a70e7d6bb180:8088/proxy/application_1579225114203_0001/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1579225114203_0001
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-01-17 02:16:11,798 Stage-1 map = 0%,  reduce = 0%
2020-01-17 02:16:18,817 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 2.8 sec
2020-01-17 02:16:27,006 Stage-1 map = 

In [11]:
%hive_quit